In [1]:
import os
import numpy as np
import pandas as pd
from scapy.utils import rdpcap, PcapReader
from scapy.layers.inet import IP, TCP, UDP

Согласно статье, на вход модели подаются первые n пакетов каждого сетевого потока, причем:

- Фильтрация: Выбираются пакеты, принадлежащие одному 5-тuple (IP-адреса, порты, протокол).

- Анонимизация: MAC- и IP-адреса маскируются.

- Трансформация: Каждый пакет приводится к фиксированной длине l (если пакет длиннее – усечение, если короче – дополнение нулями).

- Нормализация: Значения байтов нормализуются (например, делением на 255).

- Конкатенация: n пакетов объединяются в единый входной вектор.

Пример кода для обработки может выглядеть следующим образом (упрощённо):

In [2]:
def read_pcap_lazy(file_path):
    """
    Эффективное чтение pcap-файла с помощью генератора.
    """
    with PcapReader(file_path) as pcap_reader:
        for packet in pcap_reader:
            yield packet  # Возвращает пакеты по одному (ленивая загрузка)

In [3]:
# Функция для предварительной обработки одного пакета:
def preprocess_packet(packet_bytes, l=100):
    """
    Приводит пакет к фиксированной длине l.
    Если пакет длиннее l, он усекается,
    если короче — дополняется нулями.
    Затем выполняется нормализация значений.
    """
    pkt = np.frombuffer(packet_bytes, dtype=np.uint8)
    pkt = pkt[:l] if len(pkt) > l else np.pad(pkt, (0, l - len(pkt)), mode='constant')
    # Нормализация в диапазон [0, 1]
    return pkt.astype(np.float32) / 255.0

In [4]:
def group_packets_by_flow(pcap_file, label, n_packets=2, l=100, flow_timeout=120):
    """
    Загружает pcap-файл и группирует пакеты по потоку с учетом:
    - Таймаута: если время между пакетами превышает flow_timeout, поток завершается.
    - FIN-флага в TCP: поток завершается при обнаружении FIN-флага.
    - Анонимизации MAC и IP адресов.
    Для каждого потока выбираются первые n_packets, каждый пакет приводится к длине l,
    после чего пакеты конкатенируются в единый входной вектор.
    """
    flows = []  # Список завершенных потоков: (flow_key, [bytes пакетов])
    active_flows = { }  # Словарь активных потоков: flow_key -> {'packets': [...], 'last_time': ...}

    for pkt in read_pcap_lazy(pcap_file):
        if not pkt.haslayer(IP):
            continue

        # Анонимизация MAC-адресов, если есть Ethernet-слой
        if pkt.haslayer('Ether'):
            pkt.src = "00:00:00:00:00:00"
            pkt.dst = "00:00:00:00:00:00"

        # Анонимизация IP-адресов
        ip_layer = pkt[IP]
        ip_layer.src = "0.0.0.0"
        ip_layer.dst = "0.0.0.0"

        # Извлечение портов в зависимости от протокола
        sport, dport = None, None
        if pkt.haslayer(TCP):
            sport = pkt[TCP].sport
            dport = pkt[TCP].dport
        elif pkt.haslayer(UDP):
            sport = pkt[UDP].sport
            dport = pkt[UDP].dport

        proto = ip_layer.proto
        # Формирование ключа потока по 5-tuple
        flow_key = (ip_layer.src, ip_layer.dst, sport, dport, proto)
        pkt_time = pkt.time if hasattr(pkt, "time") else None

        if flow_key in active_flows:
            current_flow = active_flows[flow_key]
            # Если временной промежуток превышает flow_timeout, завершаем текущий поток
            if pkt_time is not None and (pkt_time - current_flow['last_time'] > flow_timeout):
                flows.append((flow_key, current_flow['packets']))
                active_flows[flow_key] = { 'packets': [], 'last_time': pkt_time }
            else:
                # Если пакет TCP и содержит FIN-флаг, то считаем поток завершенным
                if pkt.haslayer(TCP):
                    tcp_layer = pkt[TCP]
                    if 'F' in tcp_layer.flags:
                        current_flow['packets'].append(bytes(pkt))
                        flows.append((flow_key, current_flow['packets']))
                        active_flows[flow_key] = { 'packets': [], 'last_time': pkt_time }
                        continue
                # Иначе добавляем пакет в текущий поток
                current_flow['packets'].append(bytes(pkt))
                current_flow['last_time'] = pkt_time
        else:
            active_flows[flow_key] = { 'packets': [bytes(pkt)], 'last_time': pkt_time }

    # Завершаем оставшиеся активные потоки
    for key, flow in active_flows.items():
        if flow['packets']:
            flows.append((key, flow['packets']))

    # Обработка и сохранение в CSV
    flow_data = []
    vec_size = n_packets * l
    columns = [f'byte_{i}' for i in range(vec_size)] + ['label']

    for _, pkts in flows:
        if len(pkts) < n_packets:
            continue
        processed = [preprocess_packet(pkt, l) for pkt in pkts[:n_packets]]
        vector = np.concatenate(processed).tolist()
        vector.append(label)
        flow_data.append(vector)

    df = pd.DataFrame(flow_data, columns=columns)
    # dataset_file = f"{prefix}_n_packets={n_packets}_l={l}_label={label}.csv"
    # os.makedirs(os.path.dirname(dataset_file) or ".", exist_ok=True)
    # df.to_csv(dataset_file, index=False)

    return df

    # # Для каждого потока выбираем первые n_packets, обрабатываем и конкатенируем
    # flow_list = []
    # for key, pkts in flows:
    #     if len(pkts) < n_packets:
    #         continue
    #     processed_pkts = [preprocess_packet(pkt, l) for pkt in pkts[:n_packets]]
    #     catenated_pkts = np.concatenate(processed_pkts)
    #     print(catenated_pkts)
    #     flow_list.append(catenated_pkts)
    #
    # dataset_file = f"{prefix}_n_packets={n_packets}_l={l}_label={label}.npy"
    # # Создаем директорию, если требуется (dataset_file может содержать путь)
    # os.makedirs(os.path.dirname(dataset_file) or ".", exist_ok=True)
    # np.save(dataset_file, flow_list)
    #
    # return np.array(flow_list)

In [5]:
def process_pcaps_in_folder(pcap_folder, output_csv, label, n_packets, l):
    """
    Обрабатывает все PCAP-файлы в папке и сохраняет в один CSV.
    """
    if not os.path.exists(pcap_folder):
        return

    all_flows = []
    for pcap_file in os.listdir(pcap_folder):
        if pcap_file.endswith(".pcap"):
            pcap_path = os.path.join(pcap_folder, pcap_file)
            flows = group_packets_by_flow(pcap_file=pcap_path,
                                          label=label,
                                          n_packets=n_packets,
                                          l=l,
                                          )
            if flows is not None:
                all_flows.append(flows)

    if all_flows:
        final_df = pd.concat(all_flows, ignore_index=True)
        final_df.to_csv(output_csv, index=False)
        print(f"Processed {pcap_folder} PCAP files and saved to {output_csv}")


In [6]:
def process_attack_category(base_attacks_dir, attack_name, output_root, n_packets, l):
    """
    Обрабатывает все PCAP-файлы для конкретной атаки (attack/normal) и сохраняет CSV.
    """
    attack_dir = os.path.join(base_attacks_dir, attack_name)
    pcap_attack_dir = os.path.join(attack_dir, 'datasets_pcap', 'attack')
    pcap_normal_dir = os.path.join(attack_dir, 'datasets_pcap', 'normal')

    # Создаем выходную директорию для атаки: output_root/attack_name
    output_dir = os.path.join(output_root, attack_name)
    os.makedirs(output_dir, exist_ok=True)

    # Обработка атакующих PCAP (label=0)
    attack_csv = os.path.join(output_dir, f"{attack_name}_attack.csv")
    process_pcaps_in_folder(pcap_attack_dir, attack_csv, label=1, n_packets=n_packets, l=l)

    # Обработка нормальных PCAP (label=1)
    normal_csv = os.path.join(output_dir, f"{attack_name}_normal.csv")
    process_pcaps_in_folder(pcap_normal_dir, normal_csv, label=0, n_packets=n_packets, l=l)

In [8]:
# Параметры обработки
N_PACKETS = 5
L = 100

# Директория с атаками
BASE_ATTACKS_DIR = "../CIC-IDS-2017/attacks"

# Выходная директория
OUTPUT_ROOT = f"n_packets={N_PACKETS}_l={L}"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# Обработка всех категорий атак
for attack_category in os.listdir(BASE_ATTACKS_DIR):
    attack_path = os.path.join(BASE_ATTACKS_DIR, attack_category)
    if os.path.isdir(attack_path):
        process_attack_category(
                base_attacks_dir=BASE_ATTACKS_DIR,
                attack_name=attack_category,
                output_root=OUTPUT_ROOT,
                n_packets=N_PACKETS,
                l=L,
                )

Processed ../CIC-IDS-2017/attacks/brute-http/datasets_pcap/attack PCAP files and saved to n_packets=5_l=100/brute-http/brute-http_attack.csv
Processed ../CIC-IDS-2017/attacks/brute-http/datasets_pcap/normal PCAP files and saved to n_packets=5_l=100/brute-http/brute-http_normal.csv
Processed ../CIC-IDS-2017/attacks/brute-ftp/datasets_pcap/attack PCAP files and saved to n_packets=5_l=100/brute-ftp/brute-ftp_attack.csv
Processed ../CIC-IDS-2017/attacks/brute-ftp/datasets_pcap/normal PCAP files and saved to n_packets=5_l=100/brute-ftp/brute-ftp_normal.csv
Processed ../CIC-IDS-2017/attacks/inf-usb/datasets_pcap/attack PCAP files and saved to n_packets=5_l=100/inf-usb/inf-usb_attack.csv
Processed ../CIC-IDS-2017/attacks/inf-usb/datasets_pcap/normal PCAP files and saved to n_packets=5_l=100/inf-usb/inf-usb_normal.csv
Processed ../CIC-IDS-2017/attacks/xss/datasets_pcap/attack PCAP files and saved to n_packets=5_l=100/xss/xss_attack.csv
Processed ../CIC-IDS-2017/attacks/xss/datasets_pcap/normal